In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
        
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
notebook_path = os.path.dirname(os.path.abspath("v4.ipynb"))+"\\"
input_path = notebook_path + 'input\\'

In [3]:
train = pd.read_csv(input_path + 'train.csv', index_col='id')
test = pd.read_csv(input_path + 'test.csv', index_col='id')
train, X_test, y_train, y_test = train_test_split(train, train['target'], train_size=0.60)
X_test.drop(columns=['target'], inplace=True)

In [4]:
for i in list(train.index):
    if train['target'][i] < 5.5 or train['target'][i] > 10:
        train.drop(i, inplace=True)

In [5]:
y_train = train.pop("target")
X_train = train

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_graphviz
import pydot

regr = RandomForestRegressor(n_estimators = 500, max_depth=15, criterion='mse', min_samples_split=500, max_features='sqrt')
regr.fit(X_train, y_train)
preds = regr.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

tree = regr.estimators_[5]
export_graphviz(tree, out_file = 'tree1.dot', feature_names = list(X_train.columns), rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('tree1.dot')
graph.write_png('tree1.png')

In [7]:
from sklearn.model_selection import GridSearchCV

In [8]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 15, 30, 40],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300]
}

In [10]:
rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [ ]:
grid_search.best_params_

In [ ]:
best_grid = grid_search.best_estimator_
preds = best_grid.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

preds = .predict(test)
my_df = pd.DataFrame(list(test.index), columns=['id'])
my_df['target'] = preds
my_df.to_csv('sub.csv', index=False)